<a href="https://colab.research.google.com/github/Melisa7L/PPS/blob/main/Modelos_de_clasificacion_PPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Librerias

In [ ]:
!pip install scikit-learn

In [ ]:
!python -m spacy download es_core_news_sm

In [ ]:
!pip install click

In [ ]:
pip install pandas scikit-learn matplotlib

In [ ]:
pip install pandas spacy scikit-learn matplotlib

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
!pip install numpy

In [ ]:
!pip install spacy==3.6.0

In [ ]:
pip install seaborn

In [ ]:
pip install joblib

In [ ]:
pip install pandas spacy scikit-learn gensim matplotlib

In [ ]:
pip install nltk

In [ ]:
pip install scipy

In [ ]:
pip install tensorflow

**SVM**

In [ ]:
import pandas as pd
import re
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.decomposition import TruncatedSVD
from collections import Counter
import joblib

# Leer el archivo XLSX que ya se limpió anteriormente
df = pd.read_excel('Nuevo.xlsx')

# Lista de categorías
categorias = df['categoria'].unique()

# Crear un diccionario para mapear categorías a números
categoria_a_numero = {cat: num for num, cat in enumerate(categorias)}
# Asignar etiquetas numéricas a las categorías en el DataFrame
df['categoria_num'] = df['categoria'].map(categoria_a_numero)

# Diccionario para almacenar las palabras más frecuentes por categoría
palabras_por_categoria = {}

# Aumentar el límite de max_length para spaCy
nlp = spacy.load('es_core_news_sm')
nlp.max_length = 4000000  # Aumenta el límite a 4 millones de caracteres

for categoria in categorias:
    # Filtrar las filas que pertenecen a la categoría específica
    df_categoria = df[df['categoria'] == categoria]

    # Analizar la columna de texto_noticia para la categoría específica
    text = ' '.join(df_categoria['texto_noticia'].astype(str))

    # Normalización del texto
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()

    # Uso de spaCy para lematización y eliminación de stopwords
    doc = nlp(text)
    filtered_words = [
        token.lemma_ for token in doc
        if token.pos_ in ('NOUN', 'ADJ', 'VERB') and token.lemma_.lower() not in nlp.Defaults.stop_words
        and not token.is_digit  # Excluir números
    ]
    filtered_text = ' '.join(filtered_words)

    # Contar la frecuencia de las palabras utilizando un diccionario
    word_freq = Counter(filtered_words)

    # Guardar las palabras más frecuentes en un archivo de texto con codificación UTF-8
    with open(f'palabras_mas_frecuentes_{categoria}.txt', 'w', encoding='utf-8') as file:
        for word, freq in word_freq.most_common():
            file.write(f"{word}: {freq}\n")

    # Guardar las palabras más frecuentes en el diccionario
    palabras_por_categoria[categoria] = [word for word, freq in word_freq.most_common()]

# Cargar las palabras más frecuentes desde el archivo de texto
palabras_mas_frecuentes = list(word_freq.keys())

# Vectorización con TF-IDF
vectorizador_tfidf = TfidfVectorizer(vocabulary=palabras_mas_frecuentes)
X_tfidf = vectorizador_tfidf.fit_transform(df['texto_noticia'])

# Reducción de dimensionalidad con TruncatedSVD
n_components = 100  # Ajusta el número de componentes según tus necesidades
svd = TruncatedSVD(n_components=n_components)
X_tfidf_svd = svd.fit_transform(X_tfidf)

# Dividir los datos en conjunto de entrenamiento y prueba de manera estratificada
X_train, X_test, y_train, y_test = train_test_split(X_tfidf_svd, df['categoria_num'], test_size=0.2, random_state=42, stratify=df['categoria_num'])

# Inicializar y entrenar el clasificador SVM con un kernel polinómico
svm_classifier = SVC(kernel='poly', degree=2, C=1.0)
svm_classifier.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = svm_classifier.predict(X_test)

# Almacenar predicciones y etiquetas reales
resultados = {
    'y_true': y_test.tolist(),
    'y_pred': y_pred.tolist(),
    'palabras_clasificadas': palabras_por_categoria
}

# Agregar las predicciones al DataFrame original
df['predicted_category'] = svm_classifier.predict(X_tfidf_svd)

# Crear un diccionario inverso para mapear números de categoría a nombres de categoría
numero_a_categoria = {num: cat for cat, num in categoria_a_numero.items()}

# Mapear los números de categoría predichos de vuelta a los nombres de categoría
df['predicted_category'] = df['predicted_category'].map(numero_a_categoria)

# Guardar el DataFrame con las predicciones en un archivo Excel
df.to_excel('Nuevo_con_predicciones.xlsx', index=False)
print("Archivo 'Nuevo_con_predicciones.xlsx' guardado con las predicciones.")

# Guardar el modelo SVM en un archivo .pkl
modelo_svm_filename = 'modelo_svm4.pkl'
joblib.dump(svm_classifier, modelo_svm_filename)
print(f"Modelo SVM guardado en {modelo_svm_filename}")

# Guardar el vectorizador TF-IDF en un archivo .pkl
vectorizador_tfidf_filename = 'vectorizador_tfidf4.pkl'
joblib.dump(vectorizador_tfidf, vectorizador_tfidf_filename)
print(f"Vectorizador TF-IDF guardado en {vectorizador_tfidf_filename}")

# Guardar el modelo TruncatedSVD en un archivo .pkl
svd_filename = 'svd3.pkl'
joblib.dump(svd, svd_filename)
print(f"Modelo TruncatedSVD guardado en {svd_filename}")


**Redes Neuronales**

In [ ]:
import pandas as pd
import re
import spacy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Embedding, Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import joblib
from collections import Counter

# Leer el archivo XLSX que ya se limpió anteriormente
df = pd.read_excel('Nuevo.xlsx')

# Lista de categorías
categorias = df['categoria'].unique()

# Crear un diccionario para mapear categorías a números
le = LabelEncoder()
df['categoria_num'] = le.fit_transform(df['categoria'])

# Aumentar el límite de max_length para spaCy
nlp = spacy.load('es_core_news_sm')
nlp.max_length = 4000000  # Aumenta el límite a 4 millones de caracteres

# Tokenización y vectorización del texto
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['texto_noticia'])
sequences = tokenizer.texts_to_sequences(df['texto_noticia'])
X = pad_sequences(sequences)

# Dividir los datos en conjunto de entrenamiento y prueba de manera estratificada
X_train, X_test, y_train, y_test = train_test_split(X, to_categorical(df['categoria_num']), test_size=0.2, random_state=42, stratify=to_categorical(df['categoria_num']))

# Crear el modelo de red neuronal
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=X.shape[1]),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(categorias), activation='softmax')
])

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Almacenar el modelo de red neuronal en un archivo .h5
modelo_nn_filename = 'modelo_nn.h5'
model.save(modelo_nn_filename)
print(f"Modelo de red neuronal guardado en {modelo_nn_filename}")

# Guardar el LabelEncoder en un archivo .pkl
label_encoder_filename = 'label_encoder.pkl'
joblib.dump(le, label_encoder_filename)
print(f"LabelEncoder guardado en {label_encoder_filename}")

def cargar_modelo_y_label_encoder(modelo_archivo, label_encoder_archivo):
    # Cargar el modelo de red neuronal
    model = load_model(modelo_archivo)

    # Cargar el LabelEncoder
    label_encoder = joblib.load(label_encoder_archivo)

    return model, label_encoder

# Predecir las categorías para los datos de entrenamiento
y_train_pred = model.predict_classes(X_train)
df_train_pred = df.loc[X_train.index].copy()
df_train_pred['categoria_predicha'] = le.inverse_transform(y_train_pred)

# Predecir las categorías para los datos de prueba
y_test_pred = model.predict_classes(X_test)
df_test_pred = df.loc[X_test.index].copy()
df_test_pred['categoria_predicha'] = le.inverse_transform(y_test_pred)

# Combinar los resultados de entrenamiento y prueba
df_pred = pd.concat([df_train_pred, df_test_pred])

# Guardar el DataFrame con las predicciones en un archivo Excel
df_pred.to_excel('Nuevo_con_predicciones.xlsx', index=False)
print("Archivo 'Nuevo_con_predicciones.xlsx' guardado con las predicciones.")

def predecir_categoria(modelo, label_encoder, texto):
    # Tokenizar y vectorizar el texto
    sequence = tokenizer.texts_to_sequences([texto])
    X = pad_sequences(sequence, maxlen=X.shape[1])

    # Realizar la predicción
    prediction = modelo.predict(X)

    # Obtener la categoría predicha
    predicted_category = label_encoder.inverse_transform([prediction.argmax()])[0]

    return predicted_category


**Random Forest**

In [ ]:
import pandas as pd
import re
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.decomposition import TruncatedSVD
from collections import Counter
import joblib

# Leer el archivo XLSX que ya se limpió anteriormente
df = pd.read_excel('Nuevo.xlsx')

# Lista de categorías
categorias = df['categoria'].unique()

# Crear un diccionario para mapear categorías a números
categoria_a_numero = {cat: num for num, cat in enumerate(categorias)}
# Asignar etiquetas numéricas a las categorías en el DataFrame
df['categoria_num'] = df['categoria'].map(categoria_a_numero)

# Diccionario para almacenar las palabras más frecuentes por categoría
palabras_por_categoria = {}

# Aumentar el límite de max_length para spaCy
nlp = spacy.load('es_core_news_sm')
nlp.max_length = 4000000  # Aumenta el límite a 4 millones de caracteres

for categoria in categorias:
    # Filtrar las filas que pertenecen a la categoría específica
    df_categoria = df[df['categoria'] == categoria]

    # Analizar la columna de texto_noticia para la categoría específica
    text = ' '.join(df_categoria['texto_noticia'].astype(str))

    # Normalización del texto
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()

    # Uso de spaCy para lematización y eliminación de stopwords
    doc = nlp(text)
    filtered_words = [
        token.lemma_ for token in doc
        if token.pos_ in ('NOUN', 'ADJ', 'VERB') and token.lemma_.lower() not in nlp.Defaults.stop_words
        and not token.is_digit  # Excluir números
    ]
    filtered_text = ' '.join(filtered_words)

    # Contar la frecuencia de las palabras utilizando un diccionario
    word_freq = Counter(filtered_words)

    # Guardar las palabras más frecuentes en un archivo de texto con codificación UTF-8
    with open(f'palabras_mas_frecuentes_{categoria}.txt', 'w', encoding='utf-8') as file:
        for word, freq in word_freq.most_common():
            file.write(f"{word}: {freq}\n")

    # Guardar las palabras más frecuentes en el diccionario
    palabras_por_categoria[categoria] = [word for word, freq in word_freq.most_common()]

# Cargar las palabras más frecuentes desde el archivo de texto
palabras_mas_frecuentes = list(word_freq.keys())

# Vectorización con TF-IDF
vectorizador_tfidf = TfidfVectorizer(vocabulary=palabras_mas_frecuentes)
X_tfidf = vectorizador_tfidf.fit_transform(df['texto_noticia'])

# Reducción de dimensionalidad con TruncatedSVD
n_components = 100  # Ajusta el número de componentes según tus necesidades
svd = TruncatedSVD(n_components=n_components)
X_tfidf_svd = svd.fit_transform(X_tfidf)

# Dividir los datos en conjunto de entrenamiento y prueba de manera estratificada
X_train, X_test, y_train, y_test = train_test_split(X_tfidf_svd, df['categoria_num'], test_size=0.2, random_state=42, stratify=df['categoria_num'])

# Inicializar y entrenar el clasificador Random Forest
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = rf_classifier.predict(X_test)

# Almacenar predicciones y etiquetas reales
resultados = {
    'y_true': y_test.tolist(),
    'y_pred': y_pred.tolist(),
    'palabras_clasificadas': palabras_por_categoria
}

# Agregar las predicciones al DataFrame original
df['predicted_category'] = rf_classifier.predict(X_tfidf_svd)

# Crear un diccionario inverso para mapear números de categoría a nombres de categoría
numero_a_categoria = {num: cat for cat, num in categoria_a_numero.items()}

# Mapear los números de categoría predichos de vuelta a los nombres de categoría
df['predicted_category'] = df['predicted_category'].map(numero_a_categoria)

# Guardar el DataFrame con las predicciones en un archivo Excel
df.to_excel('Nuevo_con_predicciones.xlsx', index=False)
print("Archivo 'Nuevo_con_predicciones.xlsx' guardado con las predicciones.")

# Guardar el modelo Random Forest en un archivo .pkl
modelo_rf_filename = 'modelo_rf.pkl'
joblib.dump(rf_classifier, modelo_rf_filename)
print(f"Modelo Random Forest guardado en {modelo_rf_filename}")

# Guardar el vectorizador TF-IDF en un archivo .pkl
vectorizador_tfidf_filename = 'vectorizador_tfidf.pkl'
joblib.dump(vectorizador_tfidf, vectorizador_tfidf_filename)
print(f"Vectorizador TF-IDF guardado en {vectorizador_tfidf_filename}")

# Guardar el modelo TruncatedSVD en un archivo .pkl
svd_filename = 'svd.pkl'
joblib.dump(svd, svd_filename)
print(f"Modelo TruncatedSVD guardado en {svd_filename}")
